In [ ]:
!git clone https://github.com/zhuxinyu2468/OpenSiteRec.git

Cloning into 'OpenSiteRec'...
remote: Enumerating objects: 132, done.
remote: Counting objects: 100% (2/2), done.
remote: Compressing objects: 100% (2/2), done.
remote: Total 132 (delta 0), reused 2 (delta 0), pack-reused 130
Receiving objects: 100% (132/132), 55.92 MiB | 22.55 MiB/s, done.
Resolving deltas: 100% (32/32), done.


In [ ]:
import sys
if not 'OpenSiteRec' in sys.path:
    sys.path += ['OpenSiteRec']

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
cd ./OpenSiteRec/baseline/

/content/OpenSiteRec/baseline


In [ ]:
import pandas as pd
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from scipy.sparse import csr_matrix
import scipy.sparse as sp
from sklearn.model_selection import train_test_split

In [ ]:
pd.read_pickle('../' + 'NYC' + '/' + 'NYC' + '_KG_plus.pkl')

,ID,Name,Brand,Latitude,Longitude,cate_1,cate_2,cate_3,brand:wikidata,Region_ID,District,Region,BA_ID,BA
0,42723103,Hart Island Ferry Terminal,Hart Island Ferry Terminal,40.852182,-73.772677,transportation,tranportation_service,shipping,NaN,552,Bronx,Hart Island,38.0,210
1,349323821,Five Guys,Five Guys,40.762536,-73.976697,food_and_beverage,food_and_beverage_shop,mixed_cuisine,Q1131810,88,Manhattan,Midtown-Times Square,60.0,105
2,357544577,All Saints High School,All Saints High School,40.701224,-73.944512,education,formal_education,school,NaN,958,Brooklyn,South Williamsburg,64.0,301
3,357544886,Andrew Jackson High School,Andrew Jackson High School,40.698072,-73.746594,education,formal_education,school,Q4757463,1718,Queens,Cambria Heights,44.0,413
4,357545339,Ascension School,Ascension School,40.735691,-73.878543,education,formal_education,school,NaN,2090,Queens,Elmhurst,0.0,404
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14184,14392988,Parking Area E,Parking Area A,40.737227,-73.863039,transportation,tranportation_service,vehicle_parking,NaN,1662,Queens,Corona,0.0,404
14185,14425097,Saint Vartan Park,Saint Vartan Park,40.745281,-73.974023,sports,sports_facility,multi_purpose_sports_facility,Q49556302,60,Manhattan,Murray Hill-Kips Bay,57.0,106
14186,14706185,Northeast Bronx YMCA,YMCA,40.883165,-73.842267,sports,sports_facility,multi_purpose_sports_facility,Q157169,539,Bronx,Eastchester-Edenwald-Baychester,36.0,212
14187,14718555,Lot 4,Lot #14,40.601874,-74.146794,transportation,tranportation_service,vehicle_parking,NaN,1356,Staten Island,New Springville-Willowbrook-Bulls Head-Travis,20.0,502


In [ ]:
city='NYC'
threshold=20
df = pd.read_pickle('../' + city + '/' + city + '_KG_plus.pkl')
bvc = df['Brand'].value_counts() >= threshold
bvc = bvc[bvc > 0].index
df = df[df['Brand'].isin(bvc)]
df.reset_index(inplace=True, drop=True)

brand2id, cate12id, cate22id, cate32id = {}, {}, {}, {}
for idx, row in df.iterrows():
    brand, cate_1, cate_2, cate_3 = row['Brand'], row['cate_1'], row['cate_2'], row['cate_3']
    if brand not in brand2id.keys():
        brand2id[brand] = len(brand2id)
    if cate_1 not in cate12id.keys():
        cate12id[cate_1] = len(cate12id)
    if cate_2 not in cate22id.keys():
        cate22id[cate_2] = len(cate22id)
    if cate_3 not in cate32id.keys():
        cate32id[cate_3] = len(cate32id)
brand2id = pd.DataFrame({'Brand': list(brand2id.keys()), 'Brand_ID': list(brand2id.values())})
cate12id = pd.DataFrame({'cate_1': list(cate12id.keys()), 'Cate1_ID': list(cate12id.values())})
cate22id = pd.DataFrame({'cate_2': list(cate22id.keys()), 'Cate2_ID': list(cate22id.values())})
cate32id = pd.DataFrame({'cate_3': list(cate32id.keys()), 'Cate3_ID': list(cate32id.values())})

df = df.merge(brand2id, on=['Brand'], how='left')
df = df.merge(cate12id, on=['cate_1'], how='left')
df = df.merge(cate22id, on=['cate_2'], how='left')
df = df.merge(cate32id, on=['cate_3'], how='left')
df = df[['ID', 'Name', 'Brand_ID', 'Cate1_ID', 'Cate2_ID', 'Cate3_ID', 'Region_ID']]

print(df['Brand_ID'].max())
print(df['Region_ID'].max())

np.random.seed(42)
train_data, test_data = [], []
for i in range(df['Brand_ID'].max() + 1):
    data = df[df['Brand_ID'] == i]
    x_train, x_test, y_train, y_test = train_test_split(
        data[['Brand_ID', 'Cate1_ID', 'Cate2_ID', 'Cate3_ID']], data['Region_ID'],
        test_size=0.2, random_state=42)
    x_train['Region_ID'] = y_train
    x_test['Region_ID'] = y_test
    train_data.append(x_train)
    test_data.append(x_test)

train_data, test_data = pd.concat(train_data, axis=0), pd.concat(test_data, axis=0)
print(train_data.shape,"train_data.shape")
# print(test_data.shape,"test_data.shape")
# train_data.to_pickle('../' + city + '/split/' + 'train.pkl')
# test_data.to_pickle('../' + city + '/split/' + 'test.pkl')

74
2323
(4884, 5) train_data.shape


In [ ]:
def generate_region_id(latitude, longitude, grid_size=1):
    """
    Generate a region ID based on a grid system.

    Parameters:
    - latitude: Latitude of the location.
    - longitude: Longitude of the location.
    - grid_size: Size of each grid cell in degrees. Adjust this based on your requirements.

    Returns:
    - region_id: Unique region ID based on the grid.
    """
    # Calculate grid cell indices
    grid_lat = int(latitude / grid_size)
    grid_lon = int(longitude / grid_size)

    # Combine indices to create a unique region ID
    region_id = f"R_{grid_lat}_{grid_lon}"

    return region_id

# Example coordinates
latitude = 35.626959
longitude = 139.6723

# Generate region ID based on a 0.1-degree grid
region_id = generate_region_id(latitude, longitude, grid_size=0.05)
print(f"Region ID: {region_id}")

Region ID: R_712_2793


In [ ]:
id_all=[]
for i in range(len(df['Long'])):
    id_all.append(generate_region_id(df['Lat'][i], df['Long'][i], grid_size=0.007))
print(f"Region ID: {id_all}")
print(len(set(id_all)))

Region ID: ['R_5819_-10571', 'R_5822_-10566', 'R_5817_-10568', 'R_5819_-10572', 'R_5816_-10570', 'R_5817_-10570', 'R_5817_-10569', 'R_5818_-10572', 'R_5822_-10566', 'R_5819_-10569', 'R_5819_-10571', 'R_5819_-10572', 'R_5817_-10565', 'R_5823_-10566', 'R_5819_-10569', 'R_5826_-10564', 'R_5824_-10559', 'R_5817_-10570', 'R_5818_-10569', 'R_5818_-10571', 'R_5818_-10571', 'R_5822_-10570', 'R_5817_-10569', 'R_5818_-10571', 'R_5826_-10568', 'R_5818_-10571', 'R_5818_-10571', 'R_5817_-10569', 'R_5819_-10571', 'R_5817_-10569', 'R_5817_-10569', 'R_5822_-10567', 'R_5823_-10566', 'R_5818_-10571', 'R_5817_-10569', 'R_5816_-10566', 'R_5819_-10572', 'R_5823_-10569', 'R_5826_-10568', 'R_5818_-10568', 'R_5819_-10570', 'R_5817_-10569', 'R_5825_-10568', 'R_5817_-10570', 'R_5817_-10568', 'R_5823_-10568', 'R_5819_-10569', 'R_5817_-10570', 'R_5818_-10571', 'R_5817_-10569', 'R_5818_-10571', 'R_5818_-10571', 'R_5822_-10569', 'R_5819_-10569', 'R_5817_-10569', 'R_5818_-10568', 'R_5819_-10571', 'R_5817_-10569', 'R

In [ ]:
df['region_id']=data_ids

In [ ]:
df.head()

,Unnamed: 0,geo_id,type,Long,Lat,venue_category_id,venue_category_name,region_id
0,0,0,Point,-74.003139,40.733596,4bf58dd8d48988d1e7931735,Music Venue,689
1,1,1,Point,-73.967644,40.756368,4bf58dd8d48988d11d941735,Bar,838
2,2,2,Point,-73.981393,40.724827,4bf58dd8d48988d118941735,Bar,596
3,3,3,Point,-74.006020,40.739685,4bf58dd8d48988d10c941735,French Restaurant,690
4,4,4,Point,-73.990817,40.718363,4bf58dd8d48988d116941735,Bar,559


In [ ]:
df

,geo_id,type,Long,Lat,venue_category_id,venue_category_name,region_id
0,0,Point,-74.003139,40.733596,4bf58dd8d48988d1e7931735,Music Venue,689
1,1,Point,-73.967644,40.756368,4bf58dd8d48988d11d941735,Bar,838
2,2,Point,-73.981393,40.724827,4bf58dd8d48988d118941735,Bar,596
3,3,Point,-74.006020,40.739685,4bf58dd8d48988d10c941735,French Restaurant,690
4,4,Point,-73.990817,40.718363,4bf58dd8d48988d116941735,Bar,559
...,...,...,...,...,...,...,...
9984,9984,Point,-74.108426,40.889077,4bf58dd8d48988d124941735,Office,1489
9985,9985,Point,-73.984446,40.728397,4d954b06a243a5684965b473,Residential Building (Apartment / Condo),642
9986,9986,Point,-74.004831,40.741677,4f2a23984b9023bd5841ed2c,Moving Target,741
9987,9987,Point,-73.981934,40.757790,4bf58dd8d48988d162941735,Other Great Outdoors,840


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Example data with categorical strings
data_strings = df['venue_category_name']

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the data to numerical IDs
data_ids = label_encoder.fit_transform(data_strings)
df['cate_id']=data_ids


In [ ]:
df

,geo_id,type,Long,Lat,venue_category_id,venue_category_name,region_id,cate_id
0,0,Point,-74.003139,40.733596,4bf58dd8d48988d1e7931735,Music Venue,689,145
1,1,Point,-73.967644,40.756368,4bf58dd8d48988d11d941735,Bar,838,21
2,2,Point,-73.981393,40.724827,4bf58dd8d48988d118941735,Bar,596,21
3,3,Point,-74.006020,40.739685,4bf58dd8d48988d10c941735,French Restaurant,690,89
4,4,Point,-73.990817,40.718363,4bf58dd8d48988d116941735,Bar,559,21
...,...,...,...,...,...,...,...,...
9984,9984,Point,-74.108426,40.889077,4bf58dd8d48988d124941735,Office,1489,150
9985,9985,Point,-73.984446,40.728397,4d954b06a243a5684965b473,Residential Building (Apartment / Condo),642,172
9986,9986,Point,-74.004831,40.741677,4f2a23984b9023bd5841ed2c,Moving Target,741,142
9987,9987,Point,-73.981934,40.757790,4bf58dd8d48988d162941735,Other Great Outdoors,840,151


In [ ]:
from sklearn.preprocessing import LabelEncoder

# Example data with categorical strings
data_strings = id_all

# Initialize LabelEncoder
label_encoder = LabelEncoder()

# Fit and transform the data to numerical IDs
data_ids = label_encoder.fit_transform(data_strings)

In [ ]:
data_ids

array([689, 838, 596, ..., 741, 840, 560])

In [ ]:
print(len(set(id_all)))

30


In [ ]:
df.to_csv("df.csv")